In [30]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pathlib
import os
import io
import cv2

In [2]:
model_path = "C:/Users/asus/Desktop/Major/car_detection_model"


In [3]:
%%capture
session = tf.Session(graph=tf.Graph())
# loading the model into our session created above
tf.saved_model.loader.load(session, ['serve'], model_path)

In [4]:
def draw_boxes(height, width, box, img):
    # starting coordinates of the box
    ymin = int(max(1, (box[0] * height)))
    xmin = int(max(1, (box[1] * width)))
    # last coordinates of the box
    ymax = int(min(height, (box[2] * height)))
    xmax = int(min(width, (box[3] * width)))

    # draw a rectange using the coordinates
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (10, 255, 0), 1)

In [5]:
def predict(img_name):
    FOLDER_PATH =""
    FILE_NAME = img_name
    with io.open(os.path.join(FOLDER_PATH, FILE_NAME), 'rb') as image_file:
        img = image_file.read()

    result = session.run(['detection_boxes:0','detection_scores:0'], feed_dict={
            'encoded_image_string_tensor:0': [img]})
#     print("File {} has result {}".format("this", boxes))
    boxes = result[0][0]
    scores = result[1][0]
    # read the image with opencv  
    img = cv2.imread(FILE_NAME)
    # get the width and height of the image
    imH, imW, _ = img.shape

    for i in range(len(scores)):
    # only consider a detected object if it's probability is above 25%
        if scores[i] > 0.5:
            print("The box {} has probability {}".format(boxes[i], scores[i]))
            draw_boxes(imH, imW, boxes[i], img)

    # resize the image to fir the screen
#     new_img = cv2.resize(img, (1080, 1920))
    # show the image on the screen
    cv2.imshow("image", img)
    # as soon as any key is pressed, skip to the next line
    cv2.waitKey(0)
    
    
    return boxes

187

In [84]:
bbs = [[557, 461, 98, 67]]

In [85]:
len(bbs)

1

In [86]:
cam = cv2.VideoCapture("./11.png")
i=0
while True:
    _, im0 = cam.read()
    showCrosshair = False
    fromCenter = False
    i = i + 1
    final = im0
    final = final*0
    for slot in bbs:
        print(slot)
        x,y,w,h = slot
        i = i + 1
        slot_img = im0[y:y+h,x:x+w]
#         cv2.imshow("slot"+str(i),slot_img)
        final[y:y+h,x:x+w] = im0[y:y+h,x:x+w]
        cv2.imshow("slot"+str(i),final)
        
        
    
    cv2.imshow("Image", im0)
    
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()

[557, 461, 98, 67]


In [60]:
cv2.imshow("Image", final)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [87]:
cv2.imwrite("final.png",final)

True

In [88]:
bbs = predict("final.png")

The box [0.6376586  0.41814914 0.7289611  0.5046509 ] has probability 0.5615273714065552
